<a href="https://colab.research.google.com/github/Brendanb93/Brendanb93.github.io/blob/master/Brownlow_Datathon.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Brownlow Medal 2023 Modelling

## 1. Background
Annually Betfair runs a Datathon for the Brownlow medal, this year i trained and entered a model. There were 25 entries with mine placing 6th, winning a $100 giftcard and just missing out on the more substantial prizes in the top 5.


---


**Disclaimer:**
There may be some leaps in logic in the process below as ive just tried to compress the process I used that was spread accross several jupyter notebooks with singnificant testing/trial & error into a single document here on google colab.

## 2. Data review & clean

In [ ]:
#mounting drive to access the data files
from google.colab import drive
drive.mount('/content/drive')

%cd /content/drive/MyDrive

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
/content/drive/MyDrive


In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
pd.set_option('display.max_rows', 100)
pd.set_option('display.max_columns', 100)

In [ ]:
#Read in the brownlow data file
df = pd.read_csv("Brownlow_datathon/Brownlow_Datathon_Player_Stats_2012-2022.csv")

In [ ]:
#Lets just quickly conver the two name columns into one
df['player_name'] = df['player_first_name'].str.cat(df['player_last_name'], sep=' ')

df.drop(['player_first_name', 'player_last_name'], axis=1, inplace=True)
df.head()

,brownlow_votes,match_date,venue_name,match_id,match_home_team,match_away_team,match_round,match_home_team_score,match_away_team_score,match_margin,match_winner,player_id,player_team,player_position,guernsey_number,match_local_time,match_attendance,match_weather_temp_c,match_weather_type,kicks,marks,handballs,disposals,effective_disposals,disposal_efficiency_percentage,goals,behinds,hitouts,tackles,rebounds,inside_fifties,clearances,clangers,free_kicks_for,free_kicks_against,contested_possessions,uncontested_possessions,contested_marks,marks_inside_fifty,one_percenters,bounces,goal_assists,time_on_ground_percentage,afl_fantasy_score,centre_clearances,stoppage_clearances,score_involvements,metres_gained,turnovers,intercepts,tackles_inside_fifty,contest_def_losses,contest_def_one_on_ones,contest_off_one_on_ones,contest_off_wins,def_half_pressure_acts,effective_kicks,f50_ground_ball_gets,ground_ball_gets,hitouts_to_advantage,hitout_win_percentage,intercept_marks,marks_on_lead,pressure_acts,rating_points,ruck_contests,score_launches,shots_at_goal,spoils,player_team_result,player_is_home_team,player_team_margin,player_opposing_team,is_windy,is_raining,player_name
0,0,24/03/2012,ANZ Stadium,13960,Greater Western Sydney,Sydney,1,37,100,63,Sydney,10822,Greater Western Sydney,R,38,19:20:00,38203,22,MOSTLY_SUNNY,9,6,14,23,20,88,1,0,0,6,2,0,3,3,1,2,8,15,0,2,1,0,0,80,98,0,3,4,180,2,0,0,0,0,0,0,6,9,0,7,0,0.0,0,1,15,11.6,0,2,1,1,0,1,-63,Greater Western Sydney,0,0,James McDonald
1,0,24/03/2012,ANZ Stadium,13960,Greater Western Sydney,Sydney,1,37,100,63,Sydney,10942,Sydney,CHF,37,19:20:00,38203,22,MOSTLY_SUNNY,13,6,8,21,15,71,0,1,1,2,3,7,3,3,1,0,9,13,1,1,1,0,1,92,84,0,3,5,286,3,2,2,0,0,3,1,1,8,0,5,1,0.0,0,2,12,9.8,0,1,2,1,1,0,63,Sydney,0,0,Adam Goodes
2,0,24/03/2012,ANZ Stadium,13960,Greater Western Sydney,Sydney,1,37,100,63,Sydney,10973,Greater Western Sydney,WR,11,19:20:00,38203,22,MOSTLY_SUNNY,17,8,7,24,15,63,0,0,0,6,5,1,1,6,1,1,8,16,2,0,7,0,0,91,111,0,1,1,352,7,7,1,0,0,0,0,9,10,1,4,0,0.0,3,0,15,12.4,0,1,0,3,0,1,-63,Greater Western Sydney,0,0,Chad Cornes
3,0,24/03/2012,ANZ Stadium,13960,Greater Western Sydney,Sydney,1,37,100,63,Sydney,10988,Sydney,RR,24,19:20:00,38203,22,MOSTLY_SUNNY,12,2,13,25,15,60,1,0,1,6,0,4,5,1,1,1,11,16,0,0,5,0,0,85,97,1,4,8,296,0,1,2,0,0,0,0,4,6,5,8,0,0.0,1,0,23,15.9,0,2,1,0,1,0,63,Sydney,0,0,Jude Bolton
4,0,24/03/2012,ANZ Stadium,13960,Greater Western Sydney,Sydney,1,37,100,63,Sydney,11183,Sydney,CHB,25,19:20:00,38203,22,MOSTLY_SUNNY,12,10,3,15,15,100,0,0,0,3,1,1,0,0,1,0,8,8,3,0,5,0,0,97,85,0,0,3,228,0,13,0,1,1,0,0,5,12,0,3,0,0.0,8,0,7,16.0,0,2,0,3,1,0,63,Sydney,0,0,Ted Richards


In [ ]:
#check for nulls
df.isnull().sum()

brownlow_votes                    0
match_date                        0
venue_name                        0
match_id                          0
match_home_team                   0
match_away_team                   0
match_round                       0
match_home_team_score             0
match_away_team_score             0
match_margin                      0
match_winner                      0
player_id                         0
player_team                       0
player_position                   0
guernsey_number                   0
match_local_time                  0
match_attendance                  0
match_weather_temp_c              0
match_weather_type                0
kicks                             0
marks                             0
handballs                         0
disposals                         0
effective_disposals               0
disposal_efficiency_percentage    0
goals                             0
behinds                           0
hitouts                     

In [ ]:
#We have alot of different varaibles to look at lets check the correlation between them and brownlow_votes to see if we can narrow down some likely candidates for features
corr_matrix = df.corr()['brownlow_votes'].sort_values()
corr_matrix

The default value of numeric_only in DataFrame.corr is deprecated. In a future version, it will default to False. Select only valid columns or specify the value of numeric_only to silence this warning.


guernsey_number                  -0.101441
player_id                        -0.086535
spoils                           -0.071990
contest_def_one_on_ones          -0.067102
contest_def_losses               -0.055165
one_percenters                   -0.047570
disposal_efficiency_percentage   -0.002753
match_id                         -0.001944
ruck_contests                    -0.001877
is_raining                       -0.000260
match_weather_temp_c             -0.000088
match_round                      -0.000016
is_windy                          0.000016
match_away_team_score             0.000195
match_home_team_score             0.000248
match_margin                      0.000287
match_attendance                  0.001613
hitout_win_percentage             0.010651
intercept_marks                   0.013613
hitouts                           0.014553
hitouts_to_advantage              0.018971
player_is_home_team               0.023694
free_kicks_against                0.033725
rebounds   

In [ ]:
# Lets start with the below features
selected_features = [
    'afl_fantasy_score','def_half_pressure_acts', 'time_on_ground_percentage', 'contested_marks',
    'marks_on_lead', 'pressure_acts', 'f50_ground_ball_gets', 'tackles',
    'free_kicks_for', 'player_team_margin', 'player_team_result', 'turnovers', 'marks', 'shots_at_goal',
    'inside_fifties', 'goals', 'effective_kicks', 'ground_ball_gets', 'metres_gained',
    'clearances', 'kicks', 'score_involvements', 'effective_disposals', 'contested_possessions',
    'rating_points', 'disposals', 'spoils','contest_def_one_on_ones','free_kicks_against'
]


In [ ]:
# Some people call the Brownlow the "Midfielder's medal" so lets utilize that information and our player position column to create a feature that reinforce a bias towards midfielders
position_mapping = {
    'BPL': 1,   # Defender
    'BPR': 1,   # Defender
    'FB': 1,    # Defender
    'HBFL': 1,  # Defender
    'CHB': 1.5,   # Defender
    'HBFR': 1,  # Defender
    'HFFL': 1,  # Forward
    'CHF': 1.75,   # Forward
    'HFFR': 1,  # Forward
    'FPL': 1,   # Forward
    'FF': 1,    # Forward
    'FPR': 1,   # Forward
    'WL': 1.8,    # Midfielder
    'C': 2.25,     # Midfielder
    'WR': 2,    # Midfielder
    'RK': 2,    # Midfielder
    'RR': 2.25,    # Midfielder
    'R': 1.5,     # Midfielder
    'ROV': 2.25,   # Midfielder
    'INT': 1.75,   # Interchange
    'SUB': 0    # Sub
}

df['player_position_numeric'] = df['player_position'].replace(position_mapping)

print(df[['player_position', 'player_position_numeric']])

      player_position  player_position_numeric
0                   R                     1.50
1                 CHF                     1.75
2                  WR                     2.00
3                  RR                     2.25
4                 CHB                     1.50
...               ...                      ...
84598             CHF                     1.75
84599             FPL                     1.00
84600              WL                     1.80
84601             INT                     1.75
84602             INT                     1.75

[84603 rows x 2 columns]


2.

In [ ]:
#The competition rules state that each match probability must sum to 300% and that the log loss of each match will be averaged, for this reason a random split of the data will not be
#appropriate so lets seperate the data by years for our train/test split
var_year = 2022

# Convert 'match_date' to datetime
df['match_date'] = pd.to_datetime(df['match_date'])

# Extract year from 'match_date' and create a new feature 'match_year'
df['match_year'] = df['match_date'].dt.year

df.head()

Parsing dates in DD/MM/YYYY format when dayfirst=False (the default) was specified. This may lead to inconsistently parsed dates! Specify a format to ensure consistent parsing.


,brownlow_votes,match_date,venue_name,match_id,match_home_team,match_away_team,match_round,match_home_team_score,match_away_team_score,match_margin,match_winner,player_id,player_team,player_position,guernsey_number,match_local_time,match_attendance,match_weather_temp_c,match_weather_type,kicks,marks,handballs,disposals,effective_disposals,disposal_efficiency_percentage,goals,behinds,hitouts,tackles,rebounds,inside_fifties,clearances,clangers,free_kicks_for,free_kicks_against,contested_possessions,uncontested_possessions,contested_marks,marks_inside_fifty,one_percenters,bounces,goal_assists,time_on_ground_percentage,afl_fantasy_score,centre_clearances,stoppage_clearances,score_involvements,metres_gained,turnovers,intercepts,tackles_inside_fifty,contest_def_losses,contest_def_one_on_ones,contest_off_one_on_ones,contest_off_wins,def_half_pressure_acts,effective_kicks,f50_ground_ball_gets,ground_ball_gets,hitouts_to_advantage,hitout_win_percentage,intercept_marks,marks_on_lead,pressure_acts,rating_points,ruck_contests,score_launches,shots_at_goal,spoils,player_team_result,player_is_home_team,player_team_margin,player_opposing_team,is_windy,is_raining,player_name,player_position_numeric,match_year
0,0,2012-03-24,ANZ Stadium,13960,Greater Western Sydney,Sydney,1,37,100,63,Sydney,10822,Greater Western Sydney,R,38,19:20:00,38203,22,MOSTLY_SUNNY,9,6,14,23,20,88,1,0,0,6,2,0,3,3,1,2,8,15,0,2,1,0,0,80,98,0,3,4,180,2,0,0,0,0,0,0,6,9,0,7,0,0.0,0,1,15,11.6,0,2,1,1,0,1,-63,Greater Western Sydney,0,0,James McDonald,1.50,2012
1,0,2012-03-24,ANZ Stadium,13960,Greater Western Sydney,Sydney,1,37,100,63,Sydney,10942,Sydney,CHF,37,19:20:00,38203,22,MOSTLY_SUNNY,13,6,8,21,15,71,0,1,1,2,3,7,3,3,1,0,9,13,1,1,1,0,1,92,84,0,3,5,286,3,2,2,0,0,3,1,1,8,0,5,1,0.0,0,2,12,9.8,0,1,2,1,1,0,63,Sydney,0,0,Adam Goodes,1.75,2012
2,0,2012-03-24,ANZ Stadium,13960,Greater Western Sydney,Sydney,1,37,100,63,Sydney,10973,Greater Western Sydney,WR,11,19:20:00,38203,22,MOSTLY_SUNNY,17,8,7,24,15,63,0,0,0,6,5,1,1,6,1,1,8,16,2,0,7,0,0,91,111,0,1,1,352,7,7,1,0,0,0,0,9,10,1,4,0,0.0,3,0,15,12.4,0,1,0,3,0,1,-63,Greater Western Sydney,0,0,Chad Cornes,2.00,2012
3,0,2012-03-24,ANZ Stadium,13960,Greater Western Sydney,Sydney,1,37,100,63,Sydney,10988,Sydney,RR,24,19:20:00,38203,22,MOSTLY_SUNNY,12,2,13,25,15,60,1,0,1,6,0,4,5,1,1,1,11,16,0,0,5,0,0,85,97,1,4,8,296,0,1,2,0,0,0,0,4,6,5,8,0,0.0,1,0,23,15.9,0,2,1,0,1,0,63,Sydney,0,0,Jude Bolton,2.25,2012
4,0,2012-03-24,ANZ Stadium,13960,Greater Western Sydney,Sydney,1,37,100,63,Sydney,11183,Sydney,CHB,25,19:20:00,38203,22,MOSTLY_SUNNY,12,10,3,15,15,100,0,0,0,3,1,1,0,0,1,0,8,8,3,0,5,0,0,97,85,0,0,3,228,0,13,0,1,1,0,0,5,12,0,3,0,0.0,8,0,7,16.0,0,2,0,3,1,0,63,Sydney,0,0,Ted Richards,1.50,2012


## 2. Training the Model

In [ ]:
import sklearn
import pandas as pd
import numpy as np
import shap
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import LabelEncoder
from sklearn.metrics import log_loss, accuracy_score, classification_report
from xgboost import XGBClassifier, plot_importance


selected_features = [
    'afl_fantasy_score','def_half_pressure_acts', 'time_on_ground_percentage', 'contested_marks',
    'marks_on_lead', 'pressure_acts', 'f50_ground_ball_gets', 'tackles',
    'free_kicks_for', 'player_team_margin', 'player_team_result', 'turnovers', 'marks', 'shots_at_goal',
    'inside_fifties', 'goals', 'effective_kicks', 'ground_ball_gets', 'metres_gained',
    'clearances', 'kicks', 'score_involvements', 'effective_disposals', 'contested_possessions',
    'rating_points', 'disposals', 'spoils','contest_def_one_on_ones', 'player_position_numeric', 'free_kicks_against'
]


X = df[selected_features]
y = df['brownlow_votes']

# handle missing values
imputer = SimpleImputer(strategy='mean')
X = imputer.fit_transform(X)

classifier = XGBClassifier(
    objective='multi:softprob',
    num_class=4,
    colsample_bytree=0.8,
    gamma=0.2,
    learning_rate=0.1,
    max_depth=3,
    min_child_weight=5,
    n_estimators=200,
    random_state=42,
    reg_alpha=0.1

)

# Split the df into training and test sets based on match_year
X_train = X[(df['match_year'] < var_year) | (df['match_year'] > var_year) | (df['match_year'] != 2020)]
y_train = y[(df['match_year'] < var_year) | (df['match_year'] > var_year) | (df['match_year'] != 2020)]
X_test = X[df['match_year'] == var_year]
y_test = y[df['match_year'] == var_year]
# Train the model
classifier.fit(X_train, y_train)

# Make predictions on the test set
y_pred = classifier.predict(X_test)

# Get the predicted probabilities on the test set
y_prob = classifier.predict_proba(X_test)

# Extract Match_ID and Player_ID for the relevant rows in your df
match_ids = df.loc[df['match_year'] == var_year, 'match_id']
player_ids = df.loc[df['match_year'] == var_year, 'player_id']

# Create a dfFrame or NumPy array to hold the combined df
combined_df = []

assert len(match_ids) == len(player_ids) == len(y_prob), "Mismatch in array lengths"

# Iterate through the rows in y_prob and assign Match_ID and Player_ID
for i in range(len(match_ids)):
    match_id = match_ids.iloc[i]
    player_id = player_ids.iloc[i]
    probabilities = y_prob[i]

    # Append the combined df for this row
    combined_df.append([match_id, player_id] + probabilities.tolist())

# Convert the combined df to a dfFrame (if needed)
combined_df = pd.dfFrame(combined_df, columns=['Match_ID', 'Player_ID', 'Class_0', 'Class_1', 'Class_2', 'Class_3'])

# Calculate and print accuracy
accuracy = accuracy_score(y_test, y_pred)
print(f"Accuracy: {accuracy:.4f}")

# Calculate and print log loss
logloss = log_loss(y_test, y_prob)
print(f"Log Loss: {logloss:.4f}")

# Display the classification report
print("\nClassification Report:")
print(classification_report(y_test, y_pred))

ValueError: ignored

In [ ]:
print(X_test.shape)